In [1]:

import numpy as np
import scipy as sp
import h5py
import os

import SimData2 as sim_data
import Simulator2 as sim_system
import Uncertainty2 as unc

In [2]:

##! keys values are the columns of interest of exp data,
##! but the values are the ones used by the simulator 
##* the experimental data is read as it is presented, modfications are added later in the Simaulator object
##* gas species must be consistent with the chemical equations from the json file


schema = {
    "Pressure":     "pressure",
    "CurrentName":  "current",
    "WallTemp":     "Tw",
    "T near wall":  "Tnw", 
    "O mean":       "O",
    "N":            "N",
    "CO mean":      "CO",
    "gamma.1":      "gamma_exp"
}


###* Input files and function

reactions_file = "reactionsSimple.json"

const_dict = {
        "F0": 1.5e15,           # cm^-2
        "S0": 3e13,             # cm^-2
        
        "R": 0.00831442,        # kJ/mol*K
        "kBoltz": 1.380649e-23, # J/K
}

initial_state_dict = {'O_F': 0.1, 'O2_F':0.1 ,'O_S': 0.6, 'Vdb_S':0.1, 'Odb_S': 0.1}

###* Functions for the data transformation
def compute_flux(const_dict, exp_dict, specie, molar_mass):
    den = exp_dict.get(specie, 0.0)
    v_th = np.sqrt((8.0 * const_dict['R'] * 1000 * exp_dict['Tnw'])/(molar_mass * np.pi))
    flux = 0.25 * v_th * den * 100
    return flux


def compute_remaining_flux(const_dict, exp_dict, molar_mass): 
    den = exp_dict['N'] - exp_dict['O'] - exp_dict['CO']
    v_th = np.sqrt((8.0 * const_dict['R'] * 1000 * exp_dict['Tnw'])/(molar_mass * np.pi))
    flux = 0.25 * v_th * den * 100
    return flux

####? EavgMB data extracted from the Booth et al. 2019 paper
p_data_exp = [0.2, 0.3, 0.4, 0.5, 0.6, 0.75, 1.5]
EavgMB_data = [1.04, 0.91, 0.87, 0.83, 0.77, 0.5, 0.001]
interpolator = sp.interpolate.interp1d(p_data_exp, EavgMB_data, kind='linear', fill_value=0.001, bounds_error=False)


transformations_exp = {
    'Tw':       lambda const_dict, exp_dict: exp_dict['Tw'] + 273.15,
    'fluxO2' :  lambda const_dict, exp_dict: compute_remaining_flux(const_dict, exp_dict, 0.032),
    'fluxO' :   lambda const_dict, exp_dict: compute_flux(const_dict, exp_dict, 'O', 0.016),
    'fluxCO':   lambda const_dict, exp_dict: compute_flux(const_dict, exp_dict, 'CO', 0.028),
    'fluxCO2':  lambda const_dict, exp_dict: compute_flux(const_dict, exp_dict, 'CO2', 0.048),
    'EavgMB':   lambda const_dict, exp_dict: interpolator(exp_dict['pressure']).item(),
    'Ion':      lambda const_dict, exp_dict: 1e14 * exp_dict["current"]
}

input_folder_path = "Experimental_data_Paper"

output_folder_path = "Buffer_Data"
exp_data_file = input_folder_path + ".hdf5"

In [3]:

data_loader = sim_data.DataLoader(schema, folder_path=input_folder_path, output_file=exp_data_file, output_folder=output_folder_path)
data_loader.load_data(force_update=True)


In [4]:

exp_file = os.path.join(output_folder_path, exp_data_file)

sim = sim_system.Simulator(reactions_file, const_dict, exp_file, initial_state_dict, transformations_exp=transformations_exp)
print(sim.output_parser['rates_model'])
print(sim.output_parser['reactions_list'])

print(sim.exp_data_arr[0].keys())


['O2_F', 'O_F', 'O_S', 'Odb_S', 'V_F', 'V_S', 'Vdb_S']
[['O2', 'F'], ['O', 'F'], ['O', 'S'], ['Odb', 'S'], ['V', 'F'], ['V', 'S'], ['Vdb', 'S']]
['r_1', 'r_2', 'r_3', 'r_4', 'r_5', 'r_6', 'r_7', 'r_8', 'r_9', 'r_10', 'r_11', 'r_12', 'r_14', 'r_15', 'r_16', 'r_17', 'r_18', 'r_19', 'r_20', 'r_21', 'r_22', 'r_23', 'r_24', 'r_25', 'r_26', 'r_27']
[{'id': 1, 'left': {'V_F': 1}, 'right': {'O_F': 1}, 'rate': 'adsorption', 'gas_specie': 'O', 'model_dict': {'SF': 1.0, 'E': 0.0}, 'gamma': False}, {'id': 2, 'left': {'O_F': 1}, 'right': {'V_F': 1}, 'rate': 'desorption', 'gas_specie': '', 'model_dict': {'SF': 1.0, 'E': 30.0, 'nu_d': 1000000000000000.0}, 'gamma': False}, {'id': 3, 'left': {'V_S': 1}, 'right': {'O_S': 1}, 'rate': 'adsorption', 'gas_specie': 'O', 'model_dict': {'SF': 1.0, 'E': 0.0}, 'gamma': False}, {'id': 4, 'left': {'O_F': 1}, 'right': {'V_F': 1}, 'rate': 'recomb_ER', 'gas_specie': 'O', 'model_dict': {'SF': 1.0, 'E': 0.0}, 'gamma': True}, {'id': 5, 'left': {'O_F': 1, 'V_S': 1}, 'rig

In [5]:

if __name__ == '__main__':
    exp_file = os.path.join(output_folder_path, exp_data_file)

    sim = sim_system.Simulator(reactions_file, const_dict, exp_file, initial_state_dict, transformations_exp=transformations_exp)
    
    def func_uncertainty_dict(params):
        
        E, nu_D = params
        dict_mod_vec = [
        {"id": 1, "rate": "adsorption", "model_dict": {"E": E}},
        {"id": 8, "rate": None, "model_dict": {"nu_D": nu_D}}
        ]
        return dict_mod_vec
    
    #### truncated gaussian distribution 
    
    error_prop = unc.ErrorPropagator(sim, func_uncertainty_dict, transformations_exp=transformations_exp)
    
    model_uncert_list = [(0.0, 1.0, 0.0, np.inf),
                         [1e13, 0.1*1e13, 0.0, np.inf]]  
    
    exp_uncert_list = [('CO', 0.1, 0.0, np.inf), 
                       ('Tw', 0.01, 0.0, np.inf)]
    
    
    samples_param, gammas_array = error_prop.propagate_model_parameter_uncertainty(model_uncert_list)
    print(samples_param.shape)
    print(gammas_array.shape)
    
    
    
    samples_param2, gammas_array2 = error_prop.propagate_experimental_condition_uncertainty(exp_uncert_list)
    print(samples_param2.shape)
    print(gammas_array2.shape)
    
    
    # params_mod_arr, gammas_arr = error_prop.error_propagation_model(model_uncert_list)
    # print(params_mod_arr)
    # print(gammas_arr)
    
    # params_exp_arr, gammas_arr = error_prop.error_propagation_exp(exp_uncert_list)
    
    

['O2_F', 'O_F', 'O_S', 'Odb_S', 'V_F', 'V_S', 'Vdb_S']
[['O2', 'F'], ['O', 'F'], ['O', 'S'], ['Odb', 'S'], ['V', 'F'], ['V', 'S'], ['Vdb', 'S']]
One Model:  [{'id': 1, 'left': {'V_F': 1}, 'right': {'O_F': 1}, 'rate': 'adsorption', 'gas_specie': 'O', 'model_dict': {'SF': 1.0, 'E': 0.48812700907868445}, 'gamma': False}, {'id': 2, 'left': {'O_F': 1}, 'right': {'V_F': 1}, 'rate': 'desorption', 'gas_specie': '', 'model_dict': {'SF': 1.0, 'E': 30.0, 'nu_d': 1000000000000000.0}, 'gamma': False}, {'id': 3, 'left': {'V_S': 1}, 'right': {'O_S': 1}, 'rate': 'adsorption', 'gas_specie': 'O', 'model_dict': {'SF': 1.0, 'E': 0.0}, 'gamma': False}, {'id': 4, 'left': {'O_F': 1}, 'right': {'V_F': 1}, 'rate': 'recomb_ER', 'gas_specie': 'O', 'model_dict': {'SF': 1.0, 'E': 0.0}, 'gamma': True}, {'id': 5, 'left': {'O_F': 1, 'V_S': 1}, 'right': {'V_F': 1, 'O_S': 1}, 'rate': 'diffusion', 'gas_specie': '', 'model_dict': {'SF': 1.0, 'E': 15.0, 'nu_D': 10000000000000.0, 'factor': 0.75}, 'gamma': False}, {'id': 6,